In [124]:
import yfinance as yf
import pandas as pd
import datetime as dt

In [125]:
asset='TSLA'

In [126]:
intraday=yf.download(asset,start='2024-04-21',end='2024-04-28',interval='1m') ## prices of only 7days from last 30 days is allowed

[*********************100%%**********************]  1 of 1 completed


In [127]:
intraday

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-04-22 09:30:00-04:00,140.602005,141.350006,139.619995,140.328796,140.328796,7938147
2024-04-22 09:31:00-04:00,140.315994,141.000000,139.429993,139.580002,139.580002,1197473
2024-04-22 09:32:00-04:00,139.565002,139.880005,138.802505,139.330002,139.330002,1306517
2024-04-22 09:33:00-04:00,139.320007,139.770004,138.840607,139.443695,139.443695,981321
2024-04-22 09:34:00-04:00,139.449997,139.520004,138.809998,139.135300,139.135300,920063
...,...,...,...,...,...,...
2024-04-26 15:55:00-04:00,168.350006,168.679993,168.345001,168.500000,168.500000,508934
2024-04-26 15:56:00-04:00,168.509995,168.628494,168.449997,168.490005,168.490005,323168
2024-04-26 15:57:00-04:00,168.479996,168.669998,168.339996,168.360901,168.360901,375287


In [128]:
intraday.iloc[0]

Open         1.406020e+02
High         1.413500e+02
Low          1.396200e+02
Close        1.403288e+02
Adj Close    1.403288e+02
Volume       7.938147e+06
Name: 2024-04-22 09:30:00-04:00, dtype: float64

In [129]:
def Intradaytrend(df,entry,exit):
    ret_120min=df.iloc[120].Open/df.iloc[0].Open-1  #gives me the return
    tickret=df.Open.pct_change()
    if ret_120min > entry :
        buyprice=df.iloc[121].Open
        buytime=df.iloc[121].name #I'm getting time stamp of buying time
        cumulated=(tickret.loc[buytime:]+1).cumprod()-1
        exittime= cumulated[(cumulated < -exit) | (cumulated > exit)].first_valid_index()
        if exittime==None:
            exitprice=df.iloc[-1].Open
        else:
            exitprice=df.loc[exittime + dt.timedelta(minute=1)].Open
        profit=exitprice-buyprice
        profitrel=profit/buyprice
        return profitrel
    else:
        return None

In [130]:
Intradaytrend(intraday,0.01,0.01) ## Here output should come , its not coming

In [131]:
datesframe= yf.download(asset,start='2024-03-30',end='2024-04-28')

[*********************100%%**********************]  1 of 1 completed


In [132]:
datesframe.index

DatetimeIndex(['2024-04-01', '2024-04-02', '2024-04-03', '2024-04-04',
               '2024-04-05', '2024-04-08', '2024-04-09', '2024-04-10',
               '2024-04-11', '2024-04-12', '2024-04-15', '2024-04-16',
               '2024-04-17', '2024-04-18', '2024-04-19', '2024-04-22',
               '2024-04-23', '2024-04-24', '2024-04-25', '2024-04-26'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [133]:
frames=[]

for i in datesframe.index:
    frames.append(yf.download(asset,start=i,end=i+dt.timedelta(days=1),interval='1m'))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [141]:
returns=[]

for i in frames:
    returns.append(Intradaytrend(i,0.01,0.04))

In [142]:
returns

[None,
 None,
 0.003967148612276286,
 -0.006620589891892026,
 None,
 -0.0038618833807790276,
 0.0094260640730491,
 None,
 None,
 None,
 None,
 -0.005444075480825236,
 None,
 None,
 None,
 None,
 -0.0054302537072680734,
 None,
 0.02415575799387689,
 None]

In [143]:
pd.DataFrame(returns).mean() ## play around with the parameters, here it means we are getting return of 0.23%

0    0.002313
dtype: float64